In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc
import re,os
import glob, traceback
import seaborn as sns
import matplotlib.dates as mdates

## 0. Read all csv files

In [ ]:
df = pd.read_csv('/data2/julina/scripts/tweets/cleaned_data_by_year/2019.csv')
df.drop_duplicates(subset=['text', 'created_at'], inplace=True)
df = df.loc[:, ~df.columns.str.match('Unnamed')]

df_r = pd.read_csv('/data2/julina/scripts/tweets/cleaned_data_by_year/2019_race.csv')
df_r.drop_duplicates(subset=['text', 'created_at'], inplace=True)
df_r = df_r.loc[:, ~df_r.columns.str.match('Unnamed')]

df_2019 = pd.merge(df, df_r[['id', 'user_id', 'race']],  how='left', on=['id','user_id'])
df_2019.shape

df.shape, df_r.shape, df_2019.shape

In [ ]:
df = pd.read_csv('/data2/julina/scripts/tweets/cleaned_data_by_year/2020.csv')
df.drop_duplicates(subset=['text', 'created_at'], inplace=True)
df = df.loc[:, ~df.columns.str.match('Unnamed')]

df_r = pd.read_csv('/data2/julina/scripts/tweets/cleaned_data_by_year/2020_race.csv')
df_r.drop_duplicates(subset=['text', 'created_at'], inplace=True)
df_r = df_r.loc[:, ~df_r.columns.str.match('Unnamed')]

df_2020 = pd.merge(df, df_r[['id', 'user_id', 'race']],  how='left', on=['id','user_id'])
df_2020.shape

df.shape, df_r.shape, df_2020.shape

In [ ]:
df = pd.read_csv('/data2/julina/scripts/tweets/cleaned_data_by_year/2021.csv')
df.drop_duplicates(subset=['text', 'created_at'], inplace=True)
df = df.loc[:, ~df.columns.str.match('Unnamed')]

df_r = pd.read_csv('/data2/julina/scripts/tweets/cleaned_data_by_year/2021_race.csv')
df_r.drop_duplicates(subset=['text', 'created_at'], inplace=True)
df_r = df_r.loc[:, ~df_r.columns.str.match('Unnamed')]

df_2021 = pd.merge(df, df_r[['id', 'user_id', 'race']],  how='left', on=['id','user_id'])
df_2021.shape

df.shape, df_r.shape, df_2021.shape

In [ ]:
fentanyl = ['blonde', 'fenty', 'goodfellas', 'poson', 'blues', 'freddy', 'great bear', 'birria', 'dance fever', 'fuf', 'opes', 'takeover'] + ['apache', 'China Girl', 'China Town', 'Dance Fever', 'Friend', 'Goodfellas', 'Great Bear', 'He-Man', 'Jackpot', 'King Ivory', 'Murder 8', 'Tango and Cash', 'Tango & Cash']

drug_keywords_map= {
'tobacco' : ['nicotine', 'tobacco', 'cigarette', 'cigarrette', 'cigar', 'bidis', 'snuff'],
'alcohol' : ['liquor', 'beer', 'wine'],
'cannabinoids' : ['marijuana', 'blunt', 'dope', 'ganja', 'grass', 'herb', 'joint', 'bud', 'mary jane', 'pot', 'reefer', 'green', 'trees', 'smoke', 'sinsemilla', 'skunk', 'weed','hashish', 'boom', 'gangster', 'hash', 'hash oil', 'hemp'],
'opioids' : ['heroin', 'smack', 'horse', 'brown sugar', 'dope', 
             # 'H',
             'junk', 'skag', 'skunk', 'white horse', 'China white','opium', 'laudanum', 'paregoric', 'big O', 'black stuff', 'block', 'gum', 'hop'],
'stimulants': ['cocaine', 'hydrochloride', 'blow', 'bump',
               # 'C',
                'candy', 'Charlie', 'coke', 'crack', 'flake', 'rock', 'snow', 'toot','amphetamine', 'Biphetamine', 'Dexedrine', 'bennies', 'black beauties', 'crosses', 'hearts', 'LA turnaround', 'speed', 'truck drivers', 'uppers','methamphetamine', 'Desoxyn','meth', 'ice', 'crank', 'chalk', 'crystal', 'fire', 'glass', 'go fast', 'speed'],
'club_drugs':['mdma', 'ecstasy', 'adam', 'clarity', 'eve', "lover's speed", 'peace', 'uppers','flunitrazepam', 'rohypnol', 'forget-me pill', 'mexican valium', 'r2', 'roach', 'roche', 'roffles', 'roofinol', 'rope', 'rophies','ghb', 'Gamma-hydroxybutyrate', 
              # 'G',
              'Georgia home boy', 'grievous bodily harm', 'liquid ecstasy', 'soap', 'scoop', 'goop', 'liquid X'],
'dissociative_drugs':['ketamine', 'Ketalar SV', 'cat Valium',
                      # 'K',
                      'Special K', 'vitamin K','pcp and analogs', 'phencyclidine' 'angel dust', 'boat', 'hog', 'love boat', 'peace pill','salvia divinorum', 'salvia', 'shepherdess’s herb', 'maria pastora', 'magic mint', 'sally-d','dextromethorphan', 'dxm',  'robotripping', 'robo', 'triple'],
'hallucinogens':['lsd','Lysergic acid diethylamide', 'acid', 'blotter', 'cubes', 'microdot', 'yellow sunshine', 'blue heaven', 'mescaline', 'Buttons', 'cactus', 'mesc', 'peyote', 'psilocybin', 'Magic mushrooms', 'purple passion', 'shrooms', 'little smoke'],
'other_compounds':['anabolic_steroids', 'Anadrol', 'Oxandrin', 'Durabolin', 'Depo-Testosterone', 'Equipoise', 'roids', 'juice', 'gym candy', 'pumpers','inhalants', 'Solvents', 'paint thinners', 'gasoline', 'glues', 'gasses', 'butane', 'propane', 'aerosol propellants', 'nitrous oxide',  'nitrites' ,'isoamyl', 'isobutyl', 'cyclohexyl','laughing gas', 'poppers', 'snappers', 'whippets'],
'prescription_medications':['cns_depressants', 'stimulants', 'opioid pain relievers', 'OxyContin','Oxycodone', 'Vicodin', 'Norco', 'Lortab', 'Hydrocodone', 'Acetaminophen', 'Percocet ', 'Oxycodone', 'Acetaminophen','Tramadol','Codeine','Morphine','Methadone','Demerol', 'meperidine','Acetaminophen','Tylenol', 'Excedrin', 'Vanquish','Aspirin', 'Bayer', 'Bufferin', 'Ecotrin', 'Excedrin', 'Vanquish','Diclofenac', 'Voltaren Gel','Ibuprofen', 'Advil', 'Motrin IB','Naproxen', 'Aleve']
}

drug_cols = drug_keywords_map.keys()

In [ ]:
# https://nida.nih.gov/sites/default/files/cadchart.pdf
for keyword, words in drug_keywords_map.items():
    pattern = fr'\b(?:{"|".join(words)})\b'
    # print(f'Keyword: {keyword}, Pattern: {pattern}')
    df_2019[keyword] = df_2019['text'].str.contains(pattern, case=False).astype(int)
    df_2020[keyword] = df_2020['text'].str.contains(pattern, case=False).astype(int)
    df_2021[keyword] = df_2021['text'].str.contains(pattern, case=False).astype(int)

In [ ]:
drug_cols = list(drug_keywords_map.keys())

In [ ]:
df_2019['drug_type'] = df_2019[drug_cols].apply(lambda row: next((col for col, val in zip(drug_cols, row) if val == 1), None), axis=1)
df_2020['drug_type'] = df_2020[drug_cols].apply(lambda row: next((col for col, val in zip(drug_cols, row) if val == 1), None), axis=1)
df_2021['drug_type'] = df_2021[drug_cols].apply(lambda row: next((col for col, val in zip(drug_cols, row) if val == 1), None), axis=1)

In [ ]:
# df_2019['drug_type'].value_counts(), df_2020['drug_type'].value_counts(), df_2021['drug_type'].value_counts()

In [ ]:
years = [2019, 2020, 2021]
age_groups = ['<=18','19-29', '30-39', '>=40']
race_groups = ['api', 'white', 'hispanic', 'black']
drugs_map = {
            'tobacco' :'Tobacco',
            'alcohol' : 'Alcohol',
            'cannabinoids' : 'Cannabinoids',
            'opioids' : 'Opioids',
            'stimulants' : 'Stimulants',
            'club_drugs' : 'Club drugs',
            'dissociative_drugs': 'Dissociative drugs',
            'hallucinogens' : 'Hallucinogens',
            'other_compounds' : 'Other compounds',
            'prescription_medications' : 'Prescription medications'
}

color_2019 = 'lightslategrey'
color_2020 = 'cadetblue'        # Steel Blue
color_2021 = 'steelblue' # Royal Blue
yearly_color = [color_2019, color_2020, color_2021]   # Blue, Green, Red
display_names = {
    'UserType': {
        'Person': 'Individual',
        'Org': 'Organization'
    },
    'Gender': {
        'Female': 'Female users',
        'Male': 'Male users'
    },
    'Age': {
        '<=18': '≤18',
        '19-29': '19-29',
        '30-39': '30-39',
        '>=40': '≥40'
    },
    'Race': {
        'api': 'API',
        'white': 'White',
        'hispanic': 'Hispanic',
        'black': 'Black'
    }
}

display = {'Person': 'Individual',
 'Org': 'Organization',
 'female': 'Female users',
 'male': 'Male users',
 '<=18': '≤18',
 '19-29': '19-29',
 '30-39': '30-39',
 '>=40': '≥40',
 'api': 'API',
 'white': 'White',
 'hispanic': 'Hispanic',
 'black': 'Black',
 'neu': 'Neutral',
 'pos': 'Positive',
 'neg': 'Negative'}



# 1. Statistical Distribution by demographics

In [ ]:
def get_the_agg_results(data, agg_by_user=False, race=False):
    metrics = ['Total Tweets', 'Org', 'Person', 'Female', 'Male', 
           'Teenage', 'Adult', 'Teenage Female', 'Teenage Male', 'Adult Female', 'Adult Male'] + age_groups + ['Neutral', 'Positive', 'Negative']
    if race:
        metrics += race_groups 
    
    results = {metric: [] for metric in metrics}
    for i in range(3):
        dff = data[i]
        year = years[i]
        if agg_by_user:
            dff = dff.drop_duplicates(subset=['user_id'], keep='first')
        is_org = dff[dff['org'] == 'is-org']
        non_org = dff[dff['org'] != 'is-org']

        total_tweets = dff.shape[0]
        count_is_org = is_org.shape[0]
        count_non_org = non_org.shape[0]
        percent_is_org = (count_is_org / total_tweets) * 100
        percent_non_org = (count_non_org / total_tweets) * 100

        total_user_tweets = count_non_org
        total_female_tweets = non_org[non_org['gender'] == 'female'].shape[0]
        total_male_tweets = non_org[non_org['gender'] == 'male'].shape[0]
        percent_female = (total_female_tweets / total_user_tweets) * 100
        percent_male = (total_male_tweets / total_user_tweets) * 100

        total_teenagers_tweets = non_org[non_org['is_teenage'] == 1].shape[0]
        total_adult_tweets = non_org[non_org['is_teenage'] != 1].shape[0]
        percent_teenage = (total_teenagers_tweets / total_user_tweets) * 100
        percent_adult = (total_adult_tweets / total_user_tweets) * 100

        tf = non_org[(non_org['is_teenage'] == 1) & (non_org['gender'] == 'female')].shape[0]
        tm = non_org[(non_org['is_teenage'] == 1) & (non_org['gender'] == 'male')].shape[0]
        af = non_org[(non_org['is_teenage'] != 1) & (non_org['gender'] == 'female')].shape[0]
        am = non_org[(non_org['is_teenage'] != 1) & (non_org['gender'] == 'male')].shape[0]
        percent_tf = (tf / total_user_tweets) * 100
        percent_tm = (tm / total_user_tweets) * 100
        percent_af = (af / total_user_tweets) * 100
        percent_am = (am / total_user_tweets) * 100

        total_neu_tweets = dff[(dff['sentiment'] == 'neu')].shape[0]
        total_pos_tweets = dff[(dff['sentiment'] == 'pos')].shape[0]
        total_neg_tweets = dff[(dff['sentiment'] == 'neg')].shape[0]
        percent_neu = (total_neu_tweets / total_tweets) * 100
        percent_pos = (total_pos_tweets / total_tweets) * 100
        percent_neg = (total_neg_tweets / total_tweets) * 100

        totals = {}
        percentages = {}
        for age_group in age_groups:
            totals[age_group] = non_org[non_org['age'] == age_group].shape[0]
            percentages[age_group] = (totals[age_group] / non_org.shape[0]) * 100

        if race:
            for race in race_groups:
                totals[race] = non_org[non_org['race'] == race].shape[0]
                percentages[race] = (totals[race] / non_org.shape[0]) * 100

        results['Total Tweets'].append(total_tweets)
        results['Org'].append(f"{count_is_org} ({percent_is_org:.2f}%)")
        results['Person'].append(f"{count_non_org} ({percent_non_org:.2f}%)")
        # results['Non Org User Tweets'].append(total_user_tweets)
        results['Female'].append(f"{total_female_tweets} ({percent_female:.2f}%)")
        results['Male'].append(f"{total_male_tweets} ({percent_male:.2f}%)")
        results['Teenage'].append(f"{total_teenagers_tweets} ({percent_teenage:.2f}%)")
        results['Adult'].append(f"{total_adult_tweets} ({percent_adult:.2f}%)")
        results['Teenage Female'].append(f"{tf} ({percent_tf:.2f}%)")
        results['Teenage Male'].append(f"{tm} ({percent_tm:.2f}%)")
        results['Adult Female'].append(f"{af} ({percent_af:.2f}%)")
        results['Adult Male'].append(f"{am} ({percent_am:.2f}%)")

        results['Neutral'].append(f"{total_neu_tweets} ({percent_neu:.2f}%)")
        results['Positive'].append(f"{total_pos_tweets} ({percent_pos:.2f}%)")
        results['Negative'].append(f"{total_neg_tweets} ({percent_neg:.2f}%)")

        for age_group in age_groups:
            results[f"{age_group}"].append(f"{totals[age_group]} ({percentages[age_group]:.2f}%)")

        if race:
            for race in race_groups:
                results[f"{race}"].append(f"{totals[race]} ({percentages[race]:.2f}%)")

    # Convert results to DataFrame
    results_df = pd.DataFrame(results, index=years).T
    # Print the pivoted DataFrame
    # print(results_df)
    return results_df

In [ ]:
results_df = get_the_agg_results([df_2019, df_2020, df_2021], agg_by_user=False, race=True)
print(results_df)
# results_df.to_csv('demo_dist_agg_by_post.csv')

In [ ]:
results_udf = get_the_agg_results([df_2019, df_2020, df_2021], agg_by_user=True, race=True)
print(results_udf)
# results_udf.to_csv('demo_dist_agg_by_user.csv')

In [ ]:
# Prepare data for gender
ut = results_udf.T[['Person', 'Org']].T
for year in ut.columns:
    ut[year] = ut[year].apply(extract_percentage)
s = ut.T

gender = results_udf.T[['Female', 'Male']].T
for year in gender.columns:
    gender[year] = gender[year].apply(extract_percentage)
s_gender = gender.T

age = results_udf.T[['<=18', '19-29', '30-39', '>=40']].T
for year in age.columns:
    age[year] = age[year].apply(extract_percentage)
s_age = age.T

race= results_udf.T [['api', 'white', 'hispanic', 'black']].T
for year in race.columns:
    race[year] = race[year].apply(extract_percentage)
s_race = race.T

fig, ( ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 3))
# fig, axs = plt.subplots(2, 2, figsize=(10, 6))  
# axs = axs.flatten()

# ax0 = axs[0]
# for g in s.columns:
#     ax0.plot(s.index, s[g], marker='o', label=g)
# ax0.set_xticks(s.index, labels = ['2019', '2020', '2021'])
# ax0.set_title('By User Type')
# ax0.set_xlabel('Year')
# ax0.set_ylabel('Percentage')
# ax0.legend(title="User Type")
# ax0.grid(False)

for g in s_gender.columns:
    ax1.plot(s_gender.index, s_gender[g], marker='o', label=display_names['Gender'].get(g, g))
ax1.set_xticks(s_gender.index, labels = ['2019', '2020', '2021'])
ax1.set_title('By gender')
ax1.set_xlabel('Year')
ax1.set_ylabel('Percentage')
ax1.legend(title="Gender")
ax1.grid(False)

for g in s_age.columns:
    ax2.plot(s_age.index, s_age[g], marker='o', label=display_names['Age'].get(g, g))
ax2.set_xticks(s_age.index, labels = ['2019', '2020', '2021'])
ax2.set_title('By age groups (years)')
ax2.set_xlabel('Year')
ax2.set_ylabel('Percentage')
ax2.legend(title="Age groups")
ax2.grid(False)

for g in s_race.columns:
    ax3.plot(s_race.index, s_race[g], marker='o', label=display_names['Race'].get(g, g))
ax3.set_xticks(s_race.index, labels = ['2019', '2020', '2021'])
ax3.set_title('By race')
ax3.set_xlabel('Year')
ax3.set_ylabel('Percentage')
ax3.legend(title="Race", fontsize=10)
ax3.grid(False)
plt.tight_layout()
plt.savefig('stat_line_plot.png', bbox_inches='tight')
plt.show()


In [ ]:
se= results_df.T [['Neutral', 'Positive', 'Negative']].T 
for year in se.columns:
    se[year] = se[year].apply(extract_percentage)
fig, ax = plt.subplots(figsize=(5,3))
bars = se.T.plot(kind='bar', stacked=True, ax=ax, color=['#bdc3c7', '#27ae60', '#c0392b'], width=0.3)
ax.set_ylabel('Percentage')
ax.legend(title='Sentiment', bbox_to_anchor=(1.32, 1))
for bar in bars.containers:
    ax.bar_label(bar, label_type='center', fontsize=8, fmt='%.2f%%', color='black')
ax.set_xticklabels( [2019, 2020, 2021], rotation=0)  
ax.set_xlabel('Year')
# plt.show()
plt.savefig('stat_sentiment.png',bbox_inches='tight')

# 2. Distribution across different drug types

In [ ]:
def get_the_dist_of_drugs_results(data, agg_by_user=False, race=False):
    # Initialize an empty dictionary to collect results
    results_dict = {}
    for i in range(3):
        year = years[i]
        dff = data[i]
        if agg_by_user:
            dff = dff.drop_duplicates(subset=['user_id'], keep='first')
        org = dff[dff['org'] == 'is-org']
        non_org = dff[dff['org'] != 'is-org']

        female = non_org[(non_org['gender'] == 'female')]
        male = non_org[(non_org['gender'] == 'male')]
        adult = non_org[(non_org['is_teenage'] == 1)]
        teenage = non_org[(non_org['is_teenage'] != 1)]

        to_value_counts = dff['drug_type'].value_counts()
        to_percentages = (to_value_counts / dff.shape[0]) * 100

        o_value_counts = org['drug_type'].value_counts()
        p_value_counts = non_org['drug_type'].value_counts()
        o_percentages = (o_value_counts / dff.shape[0]) * 100
        p_percentages = (p_value_counts /dff.shape[0]) * 100

        f_value_counts = female['drug_type'].value_counts()
        m_value_counts = male['drug_type'].value_counts()
        f_percentages = (f_value_counts / dff.shape[0]) * 100
        m_percentages = (m_value_counts / dff.shape[0]) * 100

        a_value_counts = adult['drug_type'].value_counts()
        t_value_counts = teenage['drug_type'].value_counts()
        a_percentages = (a_value_counts / dff.shape[0]) * 100
        t_percentages = (t_value_counts / dff.shape[0]) * 100

        value_counts = {}
        percentages = {}
        for age_group in age_groups:
            age_filter = non_org[non_org['age'] == age_group]
            value_counts[age_group] = age_filter['drug_type'].value_counts()
            percentages[age_group] = (value_counts[age_group] / dff.shape[0]) * 100

        if race:
            for race in race_groups:
                race_filter = non_org[non_org['race'] == race]
                value_counts[race] = race_filter['drug_type'].value_counts()
                percentages[race] = (value_counts[race] / non_org.shape[0]) * 100

        # Combine the counts and percentages into a DataFrame
        result = pd.DataFrame({
            f'Total {year}': to_value_counts.astype(int).astype(str) + ' (' + to_percentages.map('{:.2f}%'.format) + ')',        
            f'Org {year}': o_value_counts.astype(int).astype(str) + ' (' + o_percentages.map('{:.2f}%'.format) + ')',        
            f'Person {year}': p_value_counts.astype(int).astype(str) + ' (' + p_percentages.map('{:.2f}%'.format) + ')',        
            f'Female {year}': f_value_counts.astype(int).astype(str) + ' (' + f_percentages.map('{:.2f}%'.format) + ')',
            f'Male {year}': m_value_counts.astype(int).astype(str) + ' (' + m_percentages.map('{:.2f}%'.format) + ')',
            f'Adult {year}': a_value_counts.astype(int).astype(str) + ' (' + a_percentages.map('{:.2f}%'.format) + ')',
            f'Teenage {year}': t_value_counts.astype(int).astype(str) + ' (' + t_percentages.map('{:.2f}%'.format) + ')'
        })

        for age_group in age_groups:
            result[f'{age_group} {year}'] = value_counts[age_group].astype(int).astype(str) + ' (' + percentages[age_group].map('{:.2f}%'.format) + ')'

        if race:
            for race in race_groups:
                result[f'{race} {year}'] = value_counts[race].astype(int).astype(str) + ' (' + percentages[race].map('{:.2f}%'.format) + ')'
       
        # Fill NaN values with 0 for better readability
        result = result.fillna('0 (0.00%)')

        # Add the result to the dictionary
        results_dict[year] = result

    # Concatenate all results along columns
    combined_result = pd.concat(results_dict.values(), axis=1)

    # Prepare multi-index for columns
    columns = pd.MultiIndex.from_product([combined_result.index, years], names=['Drug', 'Year'])
    
    # Reorganize the DataFrame to have drug types as columns and two rows for male and female
    groups = ['Total', 'Org', 'Person', 'Female', 'Male', 'Adult', 'Teenage'] + age_groups + race_groups
    final_result = pd.DataFrame(index= groups, columns=columns)

    for drug_type in combined_result.index:
        for year in years:
            final_result[(drug_type, year)] = [

                combined_result.at[drug_type, f'Total {year}'],
                combined_result.at[drug_type, f'Org {year}'],
                combined_result.at[drug_type, f'Person {year}'],
                combined_result.at[drug_type, f'Female {year}'],
                combined_result.at[drug_type, f'Male {year}'],
                combined_result.at[drug_type, f'Teenage {year}'],
                combined_result.at[drug_type, f'Adult {year}'],
                combined_result.at[drug_type, f'<=18 {year}'],
                combined_result.at[drug_type, f'19-29 {year}'],
                combined_result.at[drug_type, f'30-39 {year}'],
                combined_result.at[drug_type, f'>=40 {year}'],
                combined_result.at[drug_type, f'api {year}'],
                combined_result.at[drug_type, f'white {year}'],
                combined_result.at[drug_type, f'black {year}'],
                combined_result.at[drug_type, f'hispanic {year}'],
                # combined_result.at[drug_type, f'Total {year}'],
            ]

    return final_result

In [ ]:
# final_results= get_the_dist_of_drugs_results([df_2019, df_2020, df_2021], agg_by_user = False, race = True)
# final_results.to_csv('drug_demo_dist_by_post.csv')

In [ ]:
final_results= get_the_dist_of_drugs_results([df_2019, df_2020, df_2021], agg_by_user = True, race = True)
# final_results.to_csv('drug_demo_dist_by_user.csv')

In [ ]:
final_results

In [ ]:
df=final_results.T
t = df.reset_index()
result_list = []
column = 'Total'
for y in years:
    temp = t[t['Year'] == y][['Drug', column]]
    temp['Year'] = y
    temp['Drug']= temp['Drug'].replace(drugs_map)
    temp[column] = temp[column].str.extract(r'(\d+)').astype(int)
    # temp[by_column] = temp[by_column].str.extract(r'(\d+)').astype(int)
    result_list.append(temp)
f = pd.concat(result_list)
r = f.pivot(index='Drug', columns='Year', values=column)
r = r.reset_index()
r.loc[column] = [column] + [2131457, 2604123, 1946742]
p = r.iloc[:, 1:].div(r.iloc[-1, 1:]) * 100
p.insert(0, 'Drug', r['Drug'])
p=  p[p['Drug'] != column]
# p

In [ ]:
sorted_df = p.sort_values(by=2021, ascending=True)
# Extract the data for plotting
drugs = sorted_df['Drug'].tolist()
data_2019 = sorted_df[2019].tolist()
data_2020 = sorted_df[2020].tolist()
data_2021 = sorted_df[2021].tolist()

fig, ax = plt.subplots(figsize=(8, 6))

# Set bar width and positions
bar_width = 0.25
positions = np.arange(len(drugs))
bars_2019 = ax.barh(positions - bar_width, data_2019, bar_width, label='2019', color=color_2019)  
bars_2020 = ax.barh(positions, data_2020, bar_width, label='2020', color=color_2020)         # Steel Blue
bars_2021 = ax.barh(positions + bar_width, data_2021, bar_width, label='2021', color=color_2021)  # Royal Blue
# Add percentage values on the bars
def add_percentage_labels(bars):
    for bar in bars:
        width = bar.get_width()
        ax.text(width, bar.get_y() + bar.get_height() / 2, f'{width:.1f}%', 
                va='center', ha='left', fontsize=8, color='black')

add_percentage_labels(bars_2019)
add_percentage_labels(bars_2020)
add_percentage_labels(bars_2021)

ax.set_xlabel('Percentage')
# ax.set_title("Drug Usership in Twitter Discourse from 2019 through 2021")
ax.set_yticks(positions)
ax.set_yticklabels(drugs)
ax.legend(loc='lower right', bbox_to_anchor=(1.0, 0.0), ncol=1 )
ax.grid(False)
plt.savefig('figures/Usership_dist.png',bbox_inches='tight')
plt.show()

In [ ]:
def prep_data(name):
    df=final_results.T
    t = df.reset_index()
    result_list = []
    for y in [2019, 2020, 2021]:
        temp = t[t['Year'] == y][['Drug', name]]
        temp['Year'] = y
        temp['Drug']= temp['Drug'].replace(drugs_map)
        temp[name] = temp[name].str.extract(r'(\d+)').astype(int)
        # temp[name] = temp[by_column].str.extract(r'(\d+)').astype(int)
        result_list.append(temp)
    f = pd.concat(result_list)
    r = f.pivot(index='Drug', columns='Year', values=name)
    r = r.reset_index()
    r.loc[name] = [name] + [2131457, 2604123, 1946742]
    p = r.iloc[:, 1:].div(r.iloc[-1, 1:]) * 100
    p.insert(0, 'Drug', r['Drug'])
    return p[p['Drug'] != name]

p1  = prep_data('Male')
p2  = prep_data('Female')
p3  = prep_data('Teenage')
p4  = prep_data('Adult')

In [ ]:
def wtf(data, filename):
    fig, axs = plt.subplots(2, 2, figsize=(14, 9))  
    axs = axs.flatten()
    # data = {'Teenager <=18': p1, 'Adults >18':p2}
    for i, k in enumerate(data.keys()):
        p = data[k]
        sorted_df = p.sort_values(by=2021, ascending=True)
        # Extract the data for plotting
        drugs = sorted_df['Drug'].tolist()
        # fig, ax = plt.subplots(figsize=(8, 6))
        ax = axs[i]
        bar_width = 0.25
        positions = np.arange(len(drugs))
        bars_2019 = ax.barh(positions + bar_width, sorted_df[2019].tolist(), bar_width, label='2019', color=color_2019)  
        bars_2020 = ax.barh(positions,  sorted_df[2020].tolist(), bar_width, label='2020', color=color_2020)         # Steel Blue
        bars_2021 = ax.barh(positions - bar_width, sorted_df[2021].tolist(), bar_width, label='2021', color=color_2021)  # Royal Blue
        def add_percentage_labels(bars):
            for bar in bars:
                width = bar.get_width()
                ax.text(width, bar.get_y() + bar.get_height() / 2, f'{width:.1f}%', va='center', ha='left', fontsize=6, color='black')

        add_percentage_labels(bars_2019)
        add_percentage_labels(bars_2020)
        add_percentage_labels(bars_2021)

        ax.set_xlabel('Percentage')
        ax.set_title(f'{k}')
        ax.set_yticks(positions)
        ax.set_yticklabels(drugs)
        ax.legend(loc='lower right', bbox_to_anchor=(1.0, 0.0), ncol=1, fontsize=6)
        ax.grid(False)
        # plt.savefig('Usership_dist.png',bbox_inches='tight')
        # plt.show()
    for j in range(2, len(axs)):
        fig.delaxes(axs[j])

    # fig.suptitle(f'Drug Usership in Twitter Discourse from 2019 through 2021', fontsize=14)
    plt.subplots_adjust(wspace=0.5, top=0.92)  
    plt.savefig('figures/' + filename+ '.png', bbox_inches='tight')
    plt.show()

wtf( {'Male users': p1, 'Female users':p2}, 'male_vs_female' )
wtf({'Teenager aged ≤18 years': p3, 'Adults aged >18 years':p4}, 'teenage_vs_adults')


In [ ]:
def plot_tables_side_by_side(tables, mailfile, layout=(1, 2)):
    fig, axs = plt.subplots(nrows=layout[0], ncols=layout[1], figsize=(12, 10))  
    axs = axs.flatten()  # Flatten the array of axes
    for ax, (table, filename) in zip(axs, tables):
        ax.axis('off') 
        ax.set_title(f'{filename}', fontsize=12)
        col_labels = [label.replace(' ', '\n') for label in table.columns]

        table_obj = ax.table(cellText=table.values, colLabels=col_labels, cellLoc='center', loc='center', bbox=[0, 0, 1, 1],  edges='closed')  
        # Style the table
        for i, key in enumerate(table_obj.get_celld().keys()):
            cell = table_obj.get_celld()[key]
            cell.set_edgecolor('white')  
            cell.set_linewidth(1)  

            if key[0] == 0:  
                cell.set_text_props(weight='bold', color='white')
                cell.set_facecolor('#003366')  # Dark blue
            else:
                cell.set_text_props(color='black')
                if key[0] % 2 == 0:
                    cell.set_facecolor('#f0f0f0')  # Light gray
                else:
                    cell.set_facecolor('#ffffff')  # White
                
            if key[1] == 0:   # First column
                cell.set_text_props(horizontalalignment='left')

        # Manually set column widths
        table_obj.auto_set_column_width([0,1,2])  # Automatically adjust width of the second and third columns
        table_obj.set_fontsize(12)
    for j in range(len(tables),len(axs)):
        fig.delaxes(axs[j])
                    
    plt.subplots_adjust(top=0.90)  
    if mailfile:
        # fig.suptitle('Drug User Demographics in 2020' , fontsize=16)
        plt.savefig(mailfile, bbox_inches='tight')
    plt.show()

def extract_table_data(final_results, cols, year=2020):
    def extract_percentage(column):
        return column.str.extract(r'(\d+\.\d+%)')
    
    df = final_results.T
    t = df.reset_index()
    t = t[t['Year'] == year][['Drug'] + cols + ['Total']].replace(drugs_map)
    for c in cols:
        t[c] = extract_percentage(t[c])
    t['Total'] = t['Total'].str.extract(r'(\d+)').astype(int)
    t = t.sort_values(by='Total', ascending=False)
    t = t.reset_index(drop=True)
    t = t[['Drug'] + cols]
    return t

# Extract table data for each table
t1 = extract_table_data(final_results, ['Teenage', 'Adult'])
t1.rename(columns={'Teenage': 'Teenager aged ≤18 years', 'Adult': 'Adults aged >18 years'}, inplace=True)
t2 = extract_table_data(final_results, ['Female', 'Male'])
t2.rename(columns={'Female': 'Female users', 'Male': 'Male users'}, inplace=True)

t3 = extract_table_data(final_results, ['<=18','19-29', '30-39', '>=40'])
t4 = extract_table_data(final_results, ['api', 'white', 'hispanic', 'black'])

In [ ]:
plot_tables_side_by_side([(t2, 'By gender'), (t1, 'By age') ], mailfile='demo_2020.png')
# plot_tables_side_by_side([(t3, 'By Age')], mailfile='')

In [ ]:
def plot_verticle(final_results, drug):
    def extract_data(final_results, drug, groups): 
        df=final_results.T
        t = df.reset_index()
        t=t[t['Drug']==drug].T
        t.reset_index()
        t = t.T
        for c in groups+['Total']:
            t[c] = t[c].str.extract(r'(\d+\.\d+%)')
        a= t[['Year', 'Total']+ groups]

        pd.options.mode.copy_on_write = True 
        a.iloc[:, 1:] = a.iloc[:, 1:].apply(lambda col: col.str.rstrip('%').astype(float))
        return a
    
    fig, axs = plt.subplots(2, 2, figsize=(10, 6))  
    axs = axs.flatten()
    xlabels = ['Gender', 'Age Group', 'Race', 'Teenagers Vs Adults']
    for k, g in enumerate([['Female', 'Male'], age_groups, race_groups, ['Teenage', 'Adult']]):
        df = extract_data(final_results, drug, g)
        bar_width = 0.25
        positions = np.arange(len(df.columns) - 1)
        for i, year in enumerate(df['Year']):
            axs[k].bar(positions + i * bar_width, df.loc[i, df.columns[1:]], bar_width, label=year, color=yearly_color[i])
            for j, value in enumerate(df.loc[i, df.columns[1:]]):
                axs[k].text(positions[j] + i * bar_width, value + 0.1, f'{value:.2f}%', ha='center', va='bottom', fontsize=5)

        axs[k].set_xlabel(xlabels[k], fontsize=8)
        axs[k].set_ylabel('Percentage', fontsize=8)
        axs[k].set_title(f'By {xlabels[k]}', fontsize=10)
        axs[k].set_xticks(positions + bar_width, df.columns[1:])
        # axs[k].legend(loc='lower right', bbox_to_anchor=(1.0, 0.0), ncol=1, fontsize=6)
        # Set legend
        # axs[k].legend()
        
    for j in range(4, len(axs)):
        fig.delaxes(axs[j])

    # fig.suptitle(f'{drugs_map[drug]}: Usership in 2019, 2020 and 2021', fontsize=14)
    plt.subplots_adjust(hspace=0.4, top=0.88)  
    plt.savefig('Alcohol_bars.png', bbox_inches='tight')
    plt.show()

plot_verticle(final_results, 'alcohol')

In [ ]:
# results_dict = {}
# result = pd.DataFrame({
#     '2019': df_2019['drug_type'].value_counts(),
#     '2020': df_2020['drug_type'].value_counts(),
#     '2021':  df_2021['drug_type'].value_counts()
#     })
# results_dict['drug_type'] = result
# result = pd.DataFrame({
#     '2019': df_2019['race'].value_counts(),
#     '2020': df_2020['race'].value_counts(),
#     '2021':  df_2021['race'].value_counts()
#     })
# results_dict['race'] = result
# combined_result = pd.concat(results_dict.values(), axis=0) #axis=1
# combined_result

## 3. Quartly Trend Analysis

In [ ]:
data= pd.concat([df_2019, df_2020, df_2021], ignore_index=True)

In [ ]:
data['created_at'] = pd.to_datetime(data['created_at'], format='%a %b %d %H:%M:%S %z %Y')

In [ ]:
data['quarter'] = data['created_at'].dt.to_period('Q')
data['month'] = data['created_at'].dt.to_period('M')

In [ ]:
data['week'] = data['created_at'].dt.to_period('W')

In [ ]:
def calculate_percentages(data, period, category_col, categories, drug_type=None):
    if drug_type:
        sub_data = data[data['drug_type']==drug_type]
        df = sub_data[[period, category_col]]
    else:   
        df = data[[period, category_col]]
            
    g = df.groupby([period, category_col]).size().unstack(fill_value=0).reset_index()
    g['total'] = data.groupby([period]).size().reset_index(name='Count')['Count']
    g['all'] = g[categories].sum(axis=1)
    g['all_pct'] = (g['all'] / g['total']) * 100
    for cat in categories:
        g[cat + '_pct'] = (g[cat] / g['total']) * 100
    return g

def calculate_ratio(data, period, category_col, categories, drug_type=None):
    if drug_type:
        sub_data = data[data['drug_type']==drug_type]
        df = sub_data[[period, category_col]]
        user_data = sub_data.drop_duplicates(subset=['user_id'], keep='first')
    else:   
        df = data[[period, category_col]]
        user_data = data.drop_duplicates(subset=['user_id'], keep='first')
            
    g = df.groupby([period, category_col]).size().unstack(fill_value=0).reset_index()
    g['total'] = data.groupby([period]).size().reset_index(name='Count')['Count']
    g['all'] = g[categories].sum(axis=1)
    g['all_pct'] = (g['all'] / g['total']) * 100
    for cat in categories:
        g[cat + '_pct'] = (g[cat] / g['total']) * 100
        
    u_g = user_data[[period, category_col]]
    u_g = u_g.groupby([period, category_col]).size().unstack(fill_value=0).reset_index()
    u_g['all'] = u_g[categories].sum(axis=1)
    u_g['all_pct'] = g['all'] / u_g['all']
    for cat in categories:
        u_g[cat+'_pct'] = g[cat]/u_g[cat] # renaming _pct for now "change it to _ratio"
    return u_g

def prepare_grouped_data(data, period, columns, drug_type=None, user_ratio =False):
    groups = {
        'age_groups': ['<=18', '19-29', '30-39', '>=40'],
        'gender_types': ['male', 'female'],
        'user_types': ['is-org', 'non-org'],
        'sentiment_types': ['neu', 'pos', 'neg'],
        'race_groups': ['api', 'white', 'hispanic', 'black'],
        # 'teen_adult': ['1', '0']
    }
    
    if user_ratio:
        grouped_data = {
            'age_g': calculate_ratio(data, period, 'age', groups['age_groups'], drug_type),
            'gender_g': calculate_ratio(data, period, 'gender', groups['gender_types'], drug_type),
            'user_g': calculate_ratio(data, period, 'org', groups['user_types'], drug_type),
            'sentiment_g': calculate_ratio(data, period, 'sentiment', groups['sentiment_types'], drug_type),
            'race_g': calculate_ratio(data, period, 'race', groups['race_groups'], drug_type),
            # 'teen_adult_g': calculate_ratio(data, period, 'is_teenage', groups['teen_adult'], drug_type)
        }
    else:
        grouped_data = {
            'age_g': calculate_percentages(data, period, 'age', groups['age_groups'], drug_type),
            'gender_g': calculate_percentages(data, period, 'gender', groups['gender_types'], drug_type),
            'user_g': calculate_percentages(data, period, 'org', groups['user_types'], drug_type),
            'sentiment_g': calculate_percentages(data, period, 'sentiment', groups['sentiment_types'], drug_type),
            'race_g': calculate_percentages(data, period, 'race', groups['race_groups'], drug_type),
            # 'teen_adult_g': calculate_percentages(data, period, 'is_teenage', groups['teen_adult'], drug_type)
        }
    return groups, grouped_data

In [ ]:
def plot_trend(groups, grouped_df, period, category, ax, drug_type=None):
    colors = ['blue', 'green', 'red', 'black']
    markers = ['^', 's', 'o', '*']
    s = 50 if period == 'month' else 100
    display = {
        'non-org': 'Individual',
        'is-org': 'Organization',
        'female': 'Female users',
        'male': 'Male users',
        '<=18': '≤18',
        '19-29': '19-29',
        '30-39': '30-39',
        '>=40': '≥40',
        'api': 'API',
        'white': 'White',
        'hispanic': 'Hispanic',
        'black': 'Black',
        'neu': 'Neutral',
        'pos': 'Positive',
        'neg': 'Negative'
        }
    if groups:
        for i in range(len(groups)):
            a = groups[i]
            c = colors[i]
            m = markers[i]
            ax.plot(grouped_df[period].index, grouped_df[a + '_pct'], color=c)
            ax.scatter(grouped_df[period].index, grouped_df[a + '_pct'], marker=m, s=s, color=c, label= display[a] if a in display.keys() else a)
    else:
        ax.plot(grouped_df[period].index, grouped_df['all_pct'], color='Orange')
        ax.scatter(grouped_df[period].index, grouped_df['all_pct'], marker='D', s=s, color='Orange', label='All posts')
  
    tick_positions = [-1, 3, 7, 11]
    tick_labels = ['2019Q1', '2020Q1', '2021Q1', '2022Q1']
    if period == 'month':
        tick_positions = [-1, 5, 11, 17, 23, 29, 35]
        tick_labels = ['2019-01', '2019-06', '2020-01', '2020-06', '2021-01', '2021-06', '2022-01']
    ax.set_xticks(tick_positions)
    ax.set_xticklabels(tick_labels)
    # ax.set_xlabel('Quarter' if period=='quarter' else 'Month', fontsize=12)
    ax.set_xlabel('Period', fontsize=12)
    ax.set_ylabel('Percentage', fontsize=12)
    ax.legend(fontsize=10)
    ax.grid(False)
    
    # ax.axvspan(3,5, color='blue', alpha=0.2)  # Adjust alpha for transparency
    start = 4 if period =='quarter' else 13
    xlim = ax.get_xlim()
    ax.axvline(start, color='gray', linestyle='--', linewidth=1.5)

In [ ]:
def plot_all_trends(groups, grouped_data, period, main_title, filename, drug_type=None):
    fig, axs = plt.subplots(3, 2, figsize=(15, 18))  # Adjust the grid size (3x2) and figure size as needed
    axs = axs.flatten()

    params = [(None, grouped_data['user_g'], 'default')] if drug_type else []
    # List of parameters for each plot
    params += [
        (groups['user_types'], grouped_data['user_g'], 'user type'),
        (groups['age_groups'], grouped_data['age_g'], 'age group (years)'),
        (groups['gender_types'], grouped_data['gender_g'], 'gender'),
        (groups['race_groups'], grouped_data['race_g'], 'race'),
        (groups['sentiment_types'], grouped_data['sentiment_g'], 'sentiment')
    ]

    # Plot each trend on a different subplot
    for i, (group, grouped_df, category) in enumerate(params):
        plot_trend(group, grouped_df, period, category, axs[i], drug_type)
        axs[i].set_title(f'Trend by {category}', fontsize=12)  # Individual subplot titles

    # Remove any empty subplots if there are fewer plots than subplots
    for j in range(len(params), len(axs)):
        fig.delaxes(axs[j])

    plt.subplots_adjust(top=0.95)  
    fig.suptitle(main_title, fontsize=16)

    plt.savefig('figures/'+filename+'_combined_plots.png', bbox_inches='tight')
    plt.show()

In [ ]:
# Main logic
period = 'month'
groups, grouped_data = prepare_grouped_data(data, period, ['age', 'gender', 'org', 'sentiment', 'race', 'is_teenage'], None)
main_title = "Prevalence of (All) drug use posts by various categories from 2019 to 2021"
plot_all_trends(groups, grouped_data, period, main_title, 'All')

In [ ]:
period = 'quarter'
user_data = data.drop_duplicates(subset=['user_id'], keep='first')
groups, grouped_data = prepare_grouped_data(user_data, period, ['age', 'gender', 'org', 'sentiment', 'race', 'is_teenage'], None)
main_title = "Prevalence of (All) drug esers by various categories from 2019 to 2021"
plot_all_trends(groups, grouped_data, period, main_title, 'All_users')

## 4. Trend in drug types

In [ ]:
# by post
for i, drug in enumerate(drugs_map.keys()):
    period = 'month'
    groups, grouped_data = prepare_grouped_data(data, period, ['age', 'gender', 'org', 'sentiment', 'race', 'is_teenage'], drug)
    main_title = f'{drugs_map[drug]}: prevalence of drug use posts by various categories from 2019 to 2021'
    plot_all_trends(groups, grouped_data, period, main_title, f'{i}_{drug}', drug)
    # break
# grouped_data

In [ ]:
# by user
for i, drug in enumerate(drugs_map.keys()):
    period = 'month'
    user_data = data.drop_duplicates(subset=['user_id'], keep='first')
    groups, grouped_data = prepare_grouped_data(user_data, period, ['age', 'gender', 'org', 'sentiment', 'race', 'is_teenage'], drug)
    main_title = f'{drugs_map[drug]}: prevalence of drug users by various categories from 2019 to 2021'
    plot_all_trends(groups, grouped_data, period, main_title, f'{i}_{drug}_user', drug)
    #break

In [ ]:
# Post and User Ratio
for i, drug in enumerate(drug_cols):
    period = 'month'
    user_data = data.drop_duplicates(subset=['user_id'], keep='first')
    groups, grouped_data = prepare_grouped_data(user_data, period, ['age', 'gender', 'org', 'sentiment', 'race', 'is_teenage'], drug, user_ratio=True)
    main_title = f'{drugs_map[drug]}: Ratio of Drug Use Posts Per User by Various Categories from 2019 through 2021'
    plot_all_trends(groups, grouped_data, period, main_title, f'{i}_{drug}_user', drug)
    # break
# grouped_data

In [ ]:
def plot_single_plot(data, period, ax, handles, labels, name):
    colors = ['red', 'green', 'blue', 'black', 'orange', 'teal', 'pink', 'purple', 'brown', 'gray', 'Magenta']
    for i,d in enumerate(drug_cols):
        c = colors[i]
        # sub_data = data[data['drug_type']==d]
        _, v = prepare_grouped_data(data, period, ['age', 'gender', 'org', 'sentiment', 'race', 'is_teenage'], d)        
        line, = ax.plot(v['gender_g'][period].index, v['gender_g']['all_pct'], color= c, label= drugs_map[d])
        handles.append(line)
        labels.append(d)
        
    tick_positions = [-1, 3, 7, 11]
    tick_labels = ['2019Q1', '2020Q1', '2021Q1', '2022Q1']
    if period == 'month' :
        tick_positions = [-1, 5, 11, 17, 23, 29, 35]
        tick_labels = ['January 2019', 'June 2019', 'January 2020', 'June 2020', 'January 2021', 'June 2021', 'January 2022']
    elif period =='week' :
        tick_positions = [-1, 25, 50, 75, 100, 125, 150]
        # tick_labels = ['2019-01', '2019-06', '2020-01', '2020-06', '2021-01', '2021-06', '2022-01']
        tick_labels = ['January 2019', 'June 2019', 'January 2020', 'June 2020', 'January 2021', 'June 2021', 'January 2022']

    ax.set_xticks(tick_positions)
    ax.set_xticklabels(tick_labels)
    ax.set_xlabel('Period', fontsize=12)
    ax.set_ylabel(f'Percentage', fontsize=12)
    ax.legend(fontsize=6)
    ax.grid(False)

    start = 60 if period =='week' else 14 
    xlim = ax.get_xlim()
    vertical_line = ax.axvline(start, color='gray', linestyle='--', linewidth=1.5)

    # dottled  = ax.legend([vertical_line], 'Pandemic Start')
    # ax.add_artist(dottled)

In [ ]:
def plot_drug_distributions(data, period):
    fig, axs = plt.subplots(2, 1, figsize=(10, 10))  # Adjust the grid size (3x2) and figure size as needed
    axs = axs.flatten()
    
    user_data = data.drop_duplicates(subset=['user_id'], keep='first')
    handles = []
    labels = []
    for i,d in enumerate([data, user_data]):
        name = 'post' if i ==0 else 'user'
        plot_single_plot(d, period, axs[i], handles, labels, name)
        axs[i].set_title(f'By {name}', fontsize=12) 

    for j in range(2, len(axs)):
        fig.delaxes(axs[j])

    plt.subplots_adjust(wspace=0.6, hspace=0.2, top=0.90)  
    # fig.suptitle("Weekly Drug Use Distribution", fontsize=14)
    # fig.legend(handles[:10], labels[:10], loc='center right', bbox_to_anchor=(0.9, 0.4), ncol=1, fontsize=8)
    plt.savefig('figures/'+period+'_post_user_distribution_normalized.png', bbox_inches='tight')
    plt.show()
 

## 5. Emotion Analysis

In [ ]:
data.columns

In [ ]:
user_data = data.drop_duplicates(subset=['user_id'], keep='first')
sub_data = user_data[user_data['drug_type']=='alcohol']
_, dff = prepare_grouped_data(user_data, 'week', ['age', 'gender', 'org', 'sentiment', 'race', 'is_teenage'], 'alcohol')
# dff['age_g']['all_pct']

In [ ]:
def plot_single_plotttt(df):
    
    df['wk'] = df['week'].astype(str)
    df['start_week'] = pd.to_datetime(df['wk'].str.split('/').str[0])
    df['year']=df['week'].dt.year
    df['week_num'] = df['start_week'].dt.isocalendar().week

    fig, ax = plt.subplots(figsize=(8, 4))
    colors = ['green', 'red', 'blue']
    for i,y in enumerate([2019, 2020, 2021]):
        sub_data = df[df['year'] == y]
        
        sub_data=sub_data[1:]
        # print(sub_data)
        ax.plot(sub_data['week_num'], sub_data['all_pct'], color=colors[i], label=str(y))
          
    ax.set_xticks([1, 13, 26, 39, 52])
    ax.set_xticklabels(['January', 'April', 'July', 'October', 'December'])
    ax.set_xlabel('Period', fontsize=12)
    ax.set_ylabel(f'Percentage', fontsize=12)
    ax.legend(fontsize=6)
    ax.grid(False)
    ax.axvline(11, color='gray', linestyle='--', linewidth=1.5)
    ax.axvline(25, color='gray', linestyle='--', linewidth=1.5)
    ax.fill_betweenx(y=[6, df['all_pct'].max()], x1=11, x2=25, color='mistyrose', alpha=0.5)
    ax.text(12.5, 6.5, 'Pandemic lockdown period', fontsize=6.5, color='black',  bbox=dict(facecolor='white', alpha=0.2))
    # ax.set_title(f'Alcohol: Weekly User distribution in 2019, 2020 and 2021', fontsize=14)
    plt.savefig('Alcohol_Comparision_normalized.png', bbox_inches='tight')
    plt.show()

In [ ]:
plot_single_plotttt(dff['age_g'] )

In [ ]:
display = {'Person': 'Individual', 'Org': 'Organization',
 'female': 'Female users', 'male': 'Male users',
 '<=18': '≤18', '19-29': '19-29', '30-39': '30-39', '>=40': '≥40',
 'api': 'API',
  'white': 'White',
  'hispanic': 'Hispanic',
  'black': 'Black',
  'neu': 'Neutral', 'pos':'Positive', 'neg': 'Negative'}

In [ ]:
pandemic_data = data[(data['created_at'] >= '2020-03-15') & (data['created_at'] <= '2020-06-15')]
user_data = pandemic_data.drop_duplicates(subset=['user_id'], keep='first')
sub_data = user_data[user_data['drug_type']=='alcohol']
_, dff = prepare_grouped_data(sub_data, 'week', ['age', 'gender', 'org', 'sentiment', 'race', 'is_teenage'], 'alcohol')

# colors = ['sienna', 'green',  'darkorange',  'blue']
fig, axs = plt.subplots(2, 2, figsize=(8, 5))  # Adjust the grid size (3x2) and figure size as needed
axs = axs.flatten()
keys = ['gender_g', 'age_g', 'race_g', 'sentiment_g']
title = ['By gender','By age group (years)', 'By race', 'By sentiment']
colors = [['crimson', 'darkblue'], ['sienna', 'green',  'darkorange',  'blue'],  ['sienna', 'green',  'darkorange',  'blue'], ['blue', 'green', 'red']]
for i, g in enumerate([['female', 'male'], age_groups, race_groups, ['neu', 'pos', 'neg']]):
    df = dff[keys[i]]
    df['wk'] = df['week'].astype(str)
    df['start_week'] = pd.to_datetime(df['wk'].str.split('/').str[0])
    df['week_num'] = df['start_week'].dt.isocalendar().week
    for j,a in enumerate(g):
        axs[i].plot(df.index, df[a+'_pct'], color=colors[i][j], label=display[a])
        # axs[i].plot(df.index, df[a+'_pct'], color=colors[i][j], label=display_names['Age'].get(g, g))
        
    axs[i].set_xticks([0, 5, 10])
    axs[i].set_xticklabels(['March 15', 'April 15', 'May 15'])
    axs[i].set_xlabel('Period', fontsize=7)
    axs[i].set_ylabel(f'Percentage', fontsize=7)
    axs[i].legend(fontsize=6)
    axs[i].grid(False)
    axs[i].set_title(f'{title[i]}', fontsize=10)

for j in range(4, len(axs)):
    fig.delaxes(axs[j])

# plt.subplots_adjust(wspace=0.6,hspace=0.4, top=0.90)  
# fig.suptitle("Alcohol: User Distribution during Peak Pandemic Lockdown Period", fontsize=14)
plt.subplots_adjust(wspace=0.6,hspace=0.5, top=0.88)  
plt.savefig('figures/Alchol_Peak_Pandemic_normalized.png', bbox_inches='tight')
plt.show()